In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import re #Support for regular expressions (RE).
import pylab as pl
from sklearn.metrics import fbeta_score

In [2]:
ct_train=pd.read_csv('H:\EDVANCER_DATA_SCIENCE_PYTHON\PROJECT_PYTHON-ML\PROJECT_2\carvan_train.csv')
ct_test=pd.read_csv('H:\EDVANCER_DATA_SCIENCE_PYTHON\PROJECT_PYTHON-ML\PROJECT_2\carvan_test.csv')

In [3]:

ct_train['data'] = 'train'
ct_test['data'] = 'test'


In [4]:
ct_test['V86'] = np.nan
ct_test = ct_test[ct_train.columns]
ct_all = pd.concat([ct_train,ct_test],axis=0)

In [5]:
ct_all.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V78,V79,V80,V81,V82,V83,V84,V85,V86,data
0,33,1,3,2,8,0,5,1,3,7,...,0,0,1,0,0,0,0,0,0.0,train
1,37,1,2,2,8,1,4,1,4,6,...,0,0,1,0,0,0,0,0,0.0,train
2,37,1,2,2,8,0,4,2,4,3,...,0,0,1,0,0,0,0,0,0.0,train
3,9,1,3,3,3,2,3,2,4,5,...,0,0,1,0,0,0,0,0,0.0,train
4,40,1,4,2,10,1,4,1,4,7,...,0,0,1,0,0,0,0,0,0.0,train


In [6]:
ct_all['V4'].unique()

array([2, 3, 1, 4, 5, 6], dtype=int64)

In [7]:
ct_all.dtypes[1:40]

V2     int64
V3     int64
V4     int64
V5     int64
V6     int64
V7     int64
V8     int64
V9     int64
V10    int64
V11    int64
V12    int64
V13    int64
V14    int64
V15    int64
V16    int64
V17    int64
V18    int64
V19    int64
V20    int64
V21    int64
V22    int64
V23    int64
V24    int64
V25    int64
V26    int64
V27    int64
V28    int64
V29    int64
V30    int64
V31    int64
V32    int64
V33    int64
V34    int64
V35    int64
V36    int64
V37    int64
V38    int64
V39    int64
V40    int64
dtype: object

In [8]:
ct_all.dtypes[40:86]

V41      int64
V42      int64
V43      int64
V44      int64
V45      int64
V46      int64
V47      int64
V48      int64
V49      int64
V50      int64
V51      int64
V52      int64
V53      int64
V54      int64
V55      int64
V56      int64
V57      int64
V58      int64
V59      int64
V60      int64
V61      int64
V62      int64
V63      int64
V64      int64
V65      int64
V66      int64
V67      int64
V68      int64
V69      int64
V70      int64
V71      int64
V72      int64
V73      int64
V74      int64
V75      int64
V76      int64
V77      int64
V78      int64
V79      int64
V80      int64
V81      int64
V82      int64
V83      int64
V84      int64
V85      int64
V86    float64
dtype: object

In [9]:
(ct_train['V86'][ct_train['V86'] == 1].value_counts()/ct_train.shape[0])*100

1    5.977327
Name: V86, dtype: float64

In [10]:
##CReating dummies for categorical vars
def dummies(data,var,freq_cutoff=0):
    t= data[var].value_counts(normalize=True) 
    t=t[t.values>freq_cutoff] 
    t=t.sort_values() 
    t_min=t.idxmin()
    t=t.drop([t_min])
    categories=t.index

    for cat in categories :
        name=var+'_'+cat
        name=re.sub(" ","",name) 
        name=re.sub("-","_",name)
        name=re.sub("\\?","Q",name) 
        name=re.sub("<","LT_",name)
        name=re.sub("\\+","",name) 
        name=re.sub("\\/","_",name) 
        name=re.sub(">","GT_",name) 
        name=re.sub("=","EQ_",name)
        name=re.sub(",","",name)
        data[name]=(data[var]==cat)+0 
               
    data=data.drop(columns=[var])
    return data

In [11]:
for i in ct_all:
    if (i!='V86' and i!='data'):
        ct_all[i] = ct_all[i].astype(str)
        ct_all = dummies(ct_all,i)

In [12]:
ct_train = ct_all[ct_all['data'] == 'train']
ct_test = ct_all[ct_all['data'] == 'test']

In [13]:
ct_test.drop(['V86','data'],axis=1,inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
del ct_train['data']

In [15]:
train1, train2 = train_test_split(ct_train,test_size=0.2,random_state=2)

In [16]:
x_train1 = train1.drop(['V86'],1)
y_train1 = train1['V86']
x_train2 = train2.drop(['V86'],1)
y_train2 = train2['V86']


In [17]:
x_train1.head()

,V1_19,V1_17,V1_16,V1_18,V1_21,V1_28,V1_20,V1_5,V1_7,V1_27,...,V82_1,V82_0,V83_3,V83_2,V83_1,V83_0,V84_1,V84_0,V85_1,V85_0
3546,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
1526,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
2490,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
4350,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
1826,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1


In [18]:
y_train1.head()

3546    0.0
1526    0.0
2490    0.0
4350    0.0
1826    0.0
Name: V86, dtype: float64

In [19]:
x_train1.reset_index(drop=True,inplace=True)
y_train1.reset_index(drop=True,inplace=True)

In [20]:
x_train1.head()

,V1_19,V1_17,V1_16,V1_18,V1_21,V1_28,V1_20,V1_5,V1_7,V1_27,...,V82_1,V82_0,V83_3,V83_2,V83_1,V83_0,V84_1,V84_0,V85_1,V85_0
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1


In [21]:
y_train1.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: V86, dtype: float64

In [22]:
svclassifier = SVC(kernel='rbf',class_weight='balanced',C=1.0,random_state=10,
                  verbose=20)
svclassifier.fit(x_train1, y_train1)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=10, shrinking=True, tol=0.001,
    verbose=20)

In [25]:
y_pred1 = svclassifier.predict(x_train1)

In [26]:
y_pred1

array([0., 0., 0., ..., 0., 0., 0.])

In [28]:
y_pred2 = svclassifier.predict(x_train2)

In [29]:
y_pred2

array([1., 1., 1., ..., 1., 0., 0.])

In [33]:
y_pred1[y_pred1 == 1].size

896

In [35]:
y_pred2[y_pred2 == 1].size

211

In [36]:
y_pred2[y_pred2 == 0].size

954

In [38]:
print(confusion_matrix(y_train1,y_pred1))
print(classification_report(y_train1,y_pred1))


[[3758  626]
 [   3  270]]
              precision    recall  f1-score   support

         0.0       1.00      0.86      0.92      4384
         1.0       0.30      0.99      0.46       273

    accuracy                           0.86      4657
   macro avg       0.65      0.92      0.69      4657
weighted avg       0.96      0.86      0.90      4657



In [41]:
print(confusion_matrix(y_train2,y_pred2))
print(classification_report(y_train2,y_pred2))

[[914 176]
 [ 40  35]]
              precision    recall  f1-score   support

         0.0       0.96      0.84      0.89      1090
         1.0       0.17      0.47      0.24        75

    accuracy                           0.81      1165
   macro avg       0.56      0.65      0.57      1165
weighted avg       0.91      0.81      0.85      1165



In [42]:
roc_auc_score(y_train1, y_pred1)

0.9231095091040347

In [43]:
roc_auc_score(y_train2, y_pred2)

0.652599388379205

In [45]:
fbeta_score(y_train1, y_pred1, beta=2)

0.6790744466800804

In [46]:
fbeta_score(y_train2, y_pred2, beta=2)

0.34246575342465757

In [47]:
ytest_pred = svclassifier.predict(ct_test)

In [48]:
ytest_pred

array([0., 1., 1., ..., 0., 1., 0.])

In [49]:
import os
os.getcwd()

'C:\\Users\\user\\Downloads'

In [51]:
ct_test.tail()

,V1_19,V1_17,V1_16,V1_18,V1_21,V1_28,V1_20,V1_5,V1_7,V1_27,...,V82_1,V82_0,V83_3,V83_2,V83_1,V83_0,V84_1,V84_0,V85_1,V85_0
3995,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
3996,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
3997,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,1
3998,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
3999,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
